In [ ]:
from robot_library_py import *
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Joy, JointState
from intera_core_msgs.msg import JointCommand

from threading import Thread
import time
import numpy as np 
from std_msgs.msg import Bool
import rotm2euler

In [ ]:
class ROSInterface:
    def __init__(self, node, robot):
        
        self.robot = robot
        self.node = node
        self.jointMap = {name: ind for ind, name in enumerate(self.robot.jointNames)} 

        self.latest_joint_states = None

    def joint_states_callback(self, msg):
        self.latest_joint_states = msg
        q = self.robot.getJoints()
        # print('joint_states:', q)
        for ind, name in enumerate(msg.name):
            if name in self.jointMap:
                q[self.jointMap[name]] = msg.position[ind]
        self.robot.setJoints(q)

    def spin_thread(self):
        self.node.create_subscription(JointState, 'robot/joint_states', self.joint_states_callback, 10) 
        rclpy.spin(self.node)

In [ ]:
rclpy.init()
robot = URDFModel('/home/ns/sawyer_robot_ros2/src/teleop_script/sawyer.urdf')
jointMap = {name: ind for ind, name in enumerate(robot.jointNames)}

node = Node('pub_dbg')
q = robot.getJoints()
ros_interface = ROSInterface(node, robot)
cmd_pub = node.create_publisher(JointCommand, '/robot/limb/right/joint_command', 10)
pass 

In [ ]:
# ros_interface.spin_thread()
t1 = Thread(target=ros_interface.spin_thread)
t1.start()

In [ ]:
msg=ros_interface.latest_joint_states
poss_prev=np.array(msg.position)

msgs=[]
motion_detected=False
stop_count=0
for i in range(1000000):
    time.sleep(0.1)
    msg=ros_interface.latest_joint_states
    msgs.append(msg)
    poss=np.array(msg.position)
    d=np.linalg.norm(poss-poss_prev)
    if np.allclose(poss_prev, poss):  # no motion
        if motion_detected:
            # print('motion stopped, d=', d)
            stop_count+=1
            if stop_count>1:
                print('stop')
                break
    elif not motion_detected:
        print('motion detected')
        motion_detected=True
        
    poss_prev=poss.copy()

print('done')
len(msgs)